In [1]:
import json
import time
import random
import pymongo
import copy
import MyCommon
from pymongo import MongoClient
from collections import deque
import cassiopeia
from cassiopeia import riotapi
import MyCommon

In [2]:
riotapi.set_region("KR")
riotapi.set_api_key("79428a9e-5d98-469b-9b9b-429c1a750d24")
riotapi.set_rate_limits((10, 10), (500, 600))
riotapi.print_calls(True)

client = MongoClient()
loladb = client.loladb73
summoners_collection = loladb.summoners
matches_collection = loladb.matches

print summoners_collection.count()

898


In [ ]:
#find different summoners based on match history
#test by adding 1000 platium summoners
summoner_count = 0
for a_idx, a_match in enumerate(matches_collection.find()):
    if a_idx < 3:
        continue
    participants = a_match['participantIdentities']
    for a_participant in participants:
        print a_participant
        a_id = a_participant['player']['summonerId']
        a_summoner = riotapi.get_summoner_by_id(a_id)
        summoners = []
        summoners.append(a_summoner)
        leagues = riotapi.get_leagues_by_summoner(summoners)
        if leagues[0][0].data.tier == "DIAMOND":#.entries
            print "Found diamond"
            for entry in leagues[0][0].data.entries:
                entry_dict = json.loads(entry.to_json())
                entry_id = entry_dict['playerOrTeamId']
                if summoners_collection.find_one({'playerOrTeamId':entry_id}) == None:
                    print "Added " + entry_id
                    summoner_count += 1
                    summoners_collection.insert_one(entry_dict)
        else:
            print "Not diamond"
    break
        
print "Added " + str(summoner_count)

{u'player': {u'profileIcon': 28, u'summonerId': 54190752, u'matchHistoryUri': u'/v1/stats/player_history/KR/208591153', u'summonerName': u'Abyss Mir'}, u'participantId': 1}
https://kr.api.pvp.net/api/lol/kr/v2.5/league/by-summoner/54190752?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
https://kr.api.pvp.net/api/lol/kr/v1.4/summoner/1354621,3439277,38032021,7873041,2796052,9218581,30061590,12005974,23506464,1733296,9148840,29932446,10382377,7913993,14022711,32781334,11478624,1331273,17701815,3073103,36441421,17726195,5004181,8612425,20692572,4035682,11057169,20781677,2793832,3917430,34561129,1821833,9246329,3303034,3424874,45660295,27601040,4046972,8872598,18662551?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
https://kr.api.pvp.net/api/lol/kr/v1.4/summoner/21950626,28021632,5791596,2062001,47920137,26860172,54801106,33561301,6864648,5737801,8305604,20800734,6850280,4642032,2770673,24950825,9613565,17602545,3679466,14330879,49750288,26211925,13580566,11046788,2804674,7783473,4293930,90245

In [37]:
entry = leagues[0][0].data.entries[0]
entry_dict = json.loads(entry.to_json())
print entry_dict['playerOrTeamId']

21864810


In [31]:
print summoners_collection.find_one()

{u'isFreshBlood': False, u'division': u'I', u'miniSeries': None, u'wins': 373, u'losses': 229, u'playerOrTeamId': u'2924890', u'latestGrabbedMatchId': 2657658633L, u'playerOrTeamName': u'\uc21c\ubc94\uc784', u'fullGrabbed': True, u'isHotStreak': False, u'isInactive': False, u'_id': ObjectId('583e9beb4b836b107c6677fc'), u'isVeteran': True, u'leaguePoints': 10}
